In [ ]:
%pylab
#%matplotlib inline
import pylab as pl
from scipy.optimize import curve_fit
from collections import deque
import sys
import pickle
from numpy import where
from scipy.interpolate import UnivariateSpline
from scipy.signal import argrelextrema



In [ ]:
def where_closest(value,array):
  abs_diff = pl.array(abs(array-value))
  wh=where(abs_diff == min(abs_diff))[0]
  wh_closest = abs_diff[wh]
  return wh_closest
where_closest(5, pl.array([435,76,90324]))

In [ ]:
def create_band(band, res, plt=False, high=0, conv=False):
  '''
  print, "create_band, band, res, /bw, /plt, high=high, /conv"
  print, "This program will create a top hat band convolved with a gaussain of sigma = res with freq data spaced from 0 to 1000 (default) GHz"
  print, "band = bandpass region in GHz"
  print, "res = freq. spacing in GHz"
  print, "/plt plot bandpass"
  print, "high=high upper GHz region"
  print, "r = create_band([140, 160], 2.0, /bw, high=500.0)"
  return, 0
  '''

  npts = pl.ceil(4000.0)
  if high : npts = pl.ceil(high*1.0/.25)
  freq = pl.arange(npts)*.25 
  response = pl.zeros(len(freq))

  inb = pl.where((freq < band[1]) & (freq > band[0]))[0]
  if band[0] == band[1] : inb = pl.where_closest(freq, band(0))[0]
  if inb[0] == -1:
    print "Band not between 0-1000 GHZ"
    return 0

  response[inb] = 1

  #let's convolve the band with our resolution. 
  if conv:
    xx = .25*pl.arange(6*res/.25+1)-3*res
    con =pl.exp(-xx**2/(2*res**2))/pl.sqrt(2*pl.pi)/res
    normalization=1./sum(abs(con))
    response = pl.convolve(response, con,'same')*normalization
  

  if plt:
    pl.figure()
    pl.plot(freq, response,'D')
    pl.xlabel('Freq(GHz)')
    pl.ylabel('Reponse')
    pl.xlim(band[0] - 3*res, band[1] + 3*res)


  result = {'Freq': freq, 'resp': response}

  return result

In [4]:
def create_interf(freq,resp,band=[], plt=False,sav=False, res=1.0, two=False):
  '''
  print, "create_interf, freq, resp, tc=tc, plt=plt,sav=sav, band=band, res=res, bw=bw, two=two"
  print, "freq, resp - put in your own frequency and response data"
  print, "/plt plots the band pass and interferrogram"
  print, "/sav saves the interferrogram to a text file"
  print, "band = band, res=res, /bw - use these to create freq/resp band with create_band"
  print, "/two - put 2 interferrograms in a row"
  return, 0
  '''

#
#  def where_closest(value,array):
#    abs_diff = pl.array(abs(array-value))
#    wh=pl.where(abs_diff == min(abs_diff))[0]
#    wh_closest = abs_diff[wh]
#    return wh_closest

  if len(band) != 0:
    r = create_band(band, res)
    freq = r['Freq']
    resp = r['resp']
    if band[1] == band[0]:
      resp = pl.zeros(len(freq))
      k = where_closest(band[0], freq)
      resp[k[0]] = 1.0

#if freq(0) != 0 then return with warning!
  if freq[0] != 0:
    print 'Must go down to zero frequency'
    return -1

  #Let's be careful with these n's
  #From DFanning
  #Let N=8, N/2 = 4, then F_ns for 0,1,2,3,4, -3,-2,-1  NOTE: no -4!

  n = pl.arange(len(freq)/2.+1)
  x = 30*n/(max(freq)-min(freq)) #30 to go from GHz to icm


  intf = pl.ifft(resp)
  x2 = pl.concatenate((x, -(x[1:len(x)-2])[::-1]))    #Crap. should this be -2 or -1
  if len(freq) % 2 == 1 : x2 = pl.concatenate((x, -(x[1:len(x)-1])[::-1])) 

  #plot, freq, resp
  #oplot, freq, FFT(intf), color=1

  if two:
    x2 = pl.concatenate((x2, x2+2*max(x2)))
    intf = pl.concatenate((intf, intf))

  q = x2.argsort()
  x2 = x2[q]
  intf = (intf[q]).real
  result ={'x': x2, 'intf':intf}

  if plt:
    if len(band) != 0 : rtemp = create_band(band, res, plt=True)
    pl.plot(freq, resp)
    pl.title('Band')
    pl.figure()
    pl.plot(x2, intf.real)
    pl.xlabel('Optical Delay (cm)')
    pl.ylabel('Response') 
    pl.title('Interferrogram')


#if sav:
#   openw, 1, sav
#   x0 = result.x(0:n_elements(x2)-1)
#   outp = [[x0], [real_part(result.intf)], [imaginary(result.intf)]]
#   printf, 1, transpose(outp)
#   close, 1

  return result


In [5]:
result=create_interf(freq=np.linspace(0,500,1000), resp=np.linspace(0,500,1000) ,band=[50,100])

In [6]:
print result.keys()

['x', 'intf']


In [ ]:
pl.plot(result['x'], result['intf'])

In [7]:
def find_intf(cts, intf, plt=False):
  intf2 = intf/max(intf)
  width = len(intf)
  conv = pl.zeros(len(cts) - width)
  #Put this in histogram form instead of for-loop form!
  for i in range(0,len(cts)-width):
    xx = cts[i:i+width] -(cts[i:i+width].mean())
    conv[i] = (sum((intf2*xx)))
  spacer = pl.zeros(pl.floor(width/2.0))
  conv = pl.concatenate((spacer, conv))
  output = {'conv': conv, 'index': pl.arange(len(conv)), 'shift': pl.floor(width/2.0)}
  if plt:
    pl.figure()
    pl.plot(output['index'], output['conv'], 'D') 
    pl.title('Convolution vs. index')
    pl.xlim(0, len(cts))
    pl.xlabel('Index')
    pl.figure()
    pl.plot(range(len(cts)),cts)
    pl.xlim(0, len (cts))
    pl.title('Time stream')
  return output


In [8]:
find_intf(np.concatenate((result['intf'], result['intf'], result['intf'])), result['intf'], plt=True)

TypeError: 'numpy.float64' object cannot be interpreted as an index

In [9]:

def find_fts_peaks_max(conv, optv, rate, thres, length, adc_cts2, onepk=False, nscans=1e6):

  number_intf = 2*nscans

  maxpk = max(conv['conv'])
  pos = pl.arange(len(conv['conv']))*optv/rate # rate is frequency of data taking
  iid = pl.arange(len(conv['conv'])) 
  conv2 = conv['conv']
  adc_cts = adc_cts2
  xax = pl.arange(len(adc_cts))
  space = .2     #in cm to look around conv. peak in the actual time stream data
               #for the "true" white light fringe
  gpk = 0

  #First find the first max of the data
  mm = pl.where(conv2 == max(conv2))[0]
  index = iid[mm[0]]
  #Look within "space" around this peak for the maximum of ADC cts
  small_reg = pl.where((pos <= (pos[mm[0]] + space)) & (pos >= (pos[mm[0]] - space)))[0]
  adc_pk = pl.where(adc_cts[small_reg] == max(adc_cts[small_reg]))[0]
  index = iid[small_reg[adc_pk[0]]]
  cindex = [iid[mm[0]]]

  if onepk: #only looking for 1 peak, so return this first one
    output = {'tpks': index, 'cpks': cindex}
    return output

  #we will looks for peaks with outliers in adc cts. Need to deal with the slope.
  rt = pl.polyfit(xax, adc_cts,1)   #first try - removing a linear fix from the whole scan
  adc_cts = adc_cts - rt[1] - rt[0]*xax
  mm = [small_reg[adc_pk[0]]]
  madc = adc_cts[mm[0]]

  while gpk < 1 :    #putting loop in to try and protect against DC spikes, only positive spikes mess up the peak finder
  #Next cut out in length around this pt
    newr = pl.where((pos < (pos[mm[0]] - length)) | (pos > (pos[mm[0]] + length)))[0]
    conv2 = conv2[newr]
    pos = pos[newr]
    iid = iid[newr]
    adc_cts = adc_cts[newr]

    #again find the peak in adc cts:
    mm = pl.where(conv2 == max(conv2))[0]
    small_reg = pl.where((pos <= (pos[mm[0]] + space)) & (pos >= (pos[mm[0]] - space)))[0]
    adc_pk = pl.where(adc_cts[small_reg] == max(adc_cts[small_reg]))[0]
    tadc_new = adc_cts[small_reg[adc_pk[0]]]

#print, conv2(mm(0)), .75*conv.conv(cindex(0))
    initial = .75*conv['conv'][cindex[0]]
    testcase = conv2[mm[0]]
    if testcase > initial:  #The first peak is Good!
       gpk = 1.0
       cindex = pl.concatenate((cindex, [iid[mm[0]]]))
       index = pl.array([index, iid[small_reg[adc_pk[0]]]])
       newr = pl.where((pos < (pos[mm[0]] - length)) | (pos > (pos[mm[0]] + length)))[0]

       if len(newr) == 0:
         #check that we are above our threshold cutoff
         accept_pks = pl.where(conv['conv'][cindex] >= thres*conv['conv'][cindex[0]])[0]
         output = {'tpks': index[accept_pks], 'cpks': cindex[accept_pks]}
         return output
       conv2 = conv2[newr]
       pos = pos[newr]
       iid = iid[newr]
       adc_cts = adc_cts[newr]
   
    if testcase < initial: #It's Bad!
      madc = tadc_new
      maxpk = max(conv2)
      cindex = iid[mm[0]]
      index = iid[small_reg[adc_pk[0]]]


  pk = max(conv2)

  while ((pk >= (thres*maxpk)) & (len(index) < number_intf)) :
    mm = pl.where(conv2 == max(conv2))[0]
    cindex = pl.concatenate((cindex, [iid[mm[0]]]))
    pk = conv2[mm[0]]
    small_reg = pl.where((pos <= (pos[mm[0]] + space)) & (pos >= (pos[mm[0]] - space)))[0]
    adc_pk = pl.where(adc_cts[small_reg] == max(adc_cts[small_reg]))[0]
    mm = [small_reg[adc_pk[0]]]
    index = pl.concatenate((index, [iid[small_reg[adc_pk[0]]]]))
    newr = pl.where((pos < (pos[mm[0]] - length)) |  (pos > (pos[mm[0]] + length)))[0]
    if len(newr) != 0:
      conv2 = conv2[newr]
      pos = pos[newr]
      iid = iid[newr]
      adc_cts = adc_cts[newr] 
    else: pk = 0   

  pts = len(index)
  #check that we are above our threshold cutoff
  accept_pks = pl.where(conv['conv'][cindex] >= thres*conv['conv'][cindex[0]])[0]
  output = {'tpks': index[accept_pks], 'cpks': cindex[accept_pks]}


  return output


In [22]:
file2=open( '../../data/raw_data/20160330_1835_90A5DHW_filter_chopped_13Hz.pkl' , 'rb')
d2=pickle.load(file2)
file2.close()
 
file1=open( '../../data/raw_data/20160330_1849_90A5DHW_ref_chopped_13Hz.pkl' , 'rb')
d1=pickle.load(file1)
file1.close()

In [23]:
print d1.keys()


X1=d1['delay0F']
Y1=d1['sig0F']
#print X1
print d1['speed']
print d1['sample freq']
print d1['max_d']
X2=d2['delay0F']
Y2=d2['sig0F']

['wlf0R', 'wlf0F', 'sig2F', 'oversample', 'ADC gain', 'wlf2F', 'scan time', 'sig2R', 'iterations', 'wlf2R', 'scan start struct_time', 'speed', 'delay1R', 'acceleration', 'max_d', 'scan1F', 'sig1R', 'max_nu', 'scan1R', 'sig1F', 'delay1F', 'run', 'wlf1R', 'delay2F', 'delay0F', 'wlf1F', 'delay2R', 'dx', 'delay0R', 'sample freq', 'scan2R', 'sig0R', 'scan0F', 'acc time', 'scan2F', 'samples requested', 'sig0F', 'scan0R']
1.0
512.032770097
25.0


In [24]:
pl.plot(X2, Y2)

In [25]:
#To do the cyclic shift, we need to find the center of the white light fringe,
#And to do that, we firstly connect all the maxima and minima
max2 = argrelextrema(Y2, np.greater)
maxspline= UnivariateSpline(X2[max2], Y2[max2], s=0)
min2 = argrelextrema(Y2, np.less)
minspline= UnivariateSpline(X2[min2], Y2[min2],s=0)
wlf= maxspline(X2)- minspline(X2)
pl.plot(wlf)

In [14]:
res = 1.0*30/(2*d1['max_d']/10)  #GHz spacing of points in the interferrogram, let's try to match resolution of the data
intf = create_interf([],[],band=[50,600], res=res)
##  if template:
##    restore, template
##    intf.x = tempx
##    intf.intf = tempy
#now need to get the right number of pts to match the data:

intf['x']=intf['x']*10.0
pt_space = d1['speed']/(1.0*d1['sample freq'])
netscale = (intf['x'][1] - intf['x'][0])/pt_space
xnew = pl.arange(pl.ceil(netscale*len(intf['x'])))*pt_space + min(intf['x'])
#print len(xnew)
intf2_func = UnivariateSpline(intf['x'],intf['intf'],s=0)
intf2=intf2_func(xnew)
regcut = 3    #tweak this around (?)

q = pl.where(abs(xnew) < regcut)   #how far out in x to take the interferrogram, make this a function of frequency
intf2 = intf2[q]
intf2 = intf2.real

#pl.plot( intf2)
#Okay, now we need to find the peaks:

conv = find_intf(-wlf, intf2, plt=False)
conv=conv['conv']
maxindex=np.where(conv==max(conv))[0]
#maxindex2=np.where(-wlf==max(-wlf))[0]
print maxindex
cyclic_Y = np.concatenate((Y2[maxindex:maxindex+int(maxindex*0.8)], Y2[maxindex-int(maxindex*0.8):maxindex])) 
cyclic_X = X2[maxindex-int(maxindex*0.8): maxindex-int(maxindex*0.8)+len(cyclic_Y)]
pl.plot(cyclic_X, cyclic_Y)
#print maxindex2

TypeError: 'numpy.float64' object cannot be interpreted as an index

In [15]:
#do the rfft

FT = np.fft.rfft(cyclic_Y)
FT_real=np.real(FT)
FT_imag=np.imag(FT)
pl.plot(abs(FT))
pl.plot(abs(FT))
FT_mag=abs(FT)
#plot(FT_mag)
maxindex=np.where(FT_mag==max(FT_mag))[0]
print maxindex
cyclic_Y2 = np.concatenate((FT_mag[maxindex:maxindex+300], FT_mag[maxindex-300:maxindex])) 
#cyclic_X2 = X2[maxindex-300: maxindex-300+len(cyclic_Y2)]


cyclic_Y2[300:]=0
cyclic_Y2[0:20]=0
pl.plot(cyclic_Y2)


demodulated=np.fft.irfft(cyclic_Y2)
x= np.linspace(min(X2), max(X2), len(demodulated))
pl.plot(x, demodulated)


NameError: name 'cyclic_Y' is not defined

In [16]:
#cyclic shift

cyclic_Y3= np.concatenate((demodulated[len(demodulated)/2:], demodulated[0:len(demodulated)/2]))
pl.plot(x,cyclic_Y3)


NameError: name 'demodulated' is not defined

In [17]:
apodized = hanning(cyclic_Y3.size)*cyclic_Y3
pl.plot(x,apodized)

NameError: name 'cyclic_Y3' is not defined